# MAIS202 Project
### Gargi Singh and Jehan Dastoor

![To make your day better :)](https://media.gettyimages.com/videos/close-up-baboon-making-noise-on-telephone-video-id712-17?s=640x640)


In [1]:
!pip install tf-nightly
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tqdm import tqdm

In [2]:
import os
os.chdir("../data")

The idea of the project is to categorize skin lesions based on the kind of cancer they are. The project will consist of a few major components:

1) Importing the data

2) Pre-processing the images by transformation (to increase the size of the dataset).

3) Creating our model, i.e. deciding our layers that we are going to use (possibly based off an already built tensorflow model).

4) Training the model

5) Testing the model using our validation data set

## 1. Importing and pre-processing the data

In [3]:
#Importing our data
from keras.models import Sequential
#Import from keras_preprocessing not from keras.preprocessing
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers

traindf = pd.read_csv("ISIC2018_Task3_Training_LesionGroupings.csv")

def append_ext(fn):
    return fn+".jpg"

traindf["image"]=traindf["image"].apply(append_ext)
datagen = ImageDataGenerator(rescale=1./255.,
                             validation_split=0.2)

train_generator = datagen.flow_from_dataframe(dataframe=traindf,
                                              directory="./ISIC2018_Task3_Training_Input/",
                                              x_col="image",
                                              y_col="diagnosis_confirm_type",
                                              subset="training",
                                              batch_size=32,
                                              color_mode="rgb",
                                              shuffle=True,
                                              class_mode="categorical",
                                              target_size=(600,450))

valid_generator = datagen.flow_from_dataframe(dataframe=traindf,
                                              directory="./ISIC2018_Task3_Training_Input/",
                                              x_col="image",
                                              y_col="diagnosis_confirm_type",
                                              subset="validation",
                                              batch_size=32,
                                              color_mode="rgb",
                                              shuffle=True,
                                              class_mode="categorical",
                                              target_size=(600,450))

Found 8012 validated image filenames belonging to 4 classes.
Found 2003 validated image filenames belonging to 4 classes.


In [4]:
#Building the model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10))

model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                6

In [ ]:
#Running the Model
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

model.fit(
        train_generator,
        steps_per_epoch=STEP_SIZE_TRAIN,
        epochs=10,
        validation_data=valid_generator,
        validation_steps=STEP_SIZE_VALID)

Epoch 1/10
